In [36]:
import os
import zipfile
import re
import subprocess
import csv
from pathlib import Path
import pandas as pd
import numpy as np
from typing import Dict, Any, Literal, List
import dotenv
import json
import yaml
import time
import glob
import hashlib
import shutil
import logging

from PIL import Image
from transformers import pipeline
import torch
import torch.nn as nn

from torch.utils.data.dataloader import DataLoader
from config import cfg
from model import make_model
from utils.re_ranking import re_ranking
from data.build_DG_dataloader import build_reid_test_loader
from processor.ori_vit_processor_with_amp import do_inference as do_inf
from processor.part_attention_vit_processor import do_inference as do_inf_pat

config_train = "config/UrbanElementsReID_train.yml"  # "config/UAM_containers.yml"
config_test = "config/UrbanElementsReID_test.yml"
#config_test = "config/UrbanElementsReID_test_reduced.yml"
competition_name = "urban-reid-challenge"
submission_message = f"DSD refinement"
experiment_id: int = int(time.time())
number_of_refinements: int = 3
base_multiplier = 1 # 1=base have the same value as each refinememnt; base_multiplier=number_of_refinements the base will have the same importance as all the refinements _combined_

################ Probably nothing has to be modified from now on ################
with open(config_train, 'r') as f:
    hyperparams_train = yaml.load(f, Loader=yaml.BaseLoader)
with open(config_test, 'r') as f:
    hyperparams_test = yaml.load(f, Loader=yaml.BaseLoader)
model_path = os.path.join(hyperparams_train['LOG_ROOT'], hyperparams_train['LOG_NAME'])

assert dotenv.load_dotenv('../../.env')
assert os.getenv('KAGGLE_USERNAME')

from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

logging.root.setLevel(logging.INFO)

if not torch.cuda.is_available():
    logging.warning("Where is your GPU dude?")

max_epoch = int(hyperparams_train['SOLVER']['MAX_EPOCHS'])
assert hyperparams_test['TEST']['WEIGHT'].split('/')[-1] == f'part_attention_vit_{max_epoch}.pth', 'not testing with the trained model...'

re_ranking_k1: int = 10 if 'reduced' in config_test  else 20
num_gallery = sum(1 for f in Path(hyperparams_test['DATASETS']['ROOT_DIR']+'/image_test').iterdir() if f.is_file())  # Expected number of images in the galary
submission_file_name = os.path.join(model_path, "track_submission.csv")
track: str = os.path.join(model_path, "track.txt")

In [43]:
hyperparams_test['DATASETS']['ROOT_DIR']

'assets/datasets/urban-reid-challenge'

In [49]:
!zip -r 

/bin/sh: 1: zip: not found


In [48]:
!apt install zip -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  unzip
The following NEW packages will be installed:
  unzip zip
0 upgraded, 2 newly installed, 0 to remove and 0 not upgraded.
Need to get 350 kB of archives.
After this operation, 930 kB of additional disk space will be used.
Err:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 unzip amd64 6.0-26ubuntu3.2
  Could not open file /var/cache/apt/archives/partial/unzip_6.0-26ubuntu3.2_amd64.deb - open (30: Read-only file system) [IP: 91.189.91.81 80]
Err:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 zip amd64 3.0-12build2
  Could not open file /var/cache/apt/archives/partial/zip_3.0-12build2_amd64.deb - open (30: Read-only file system) [IP: 185.125.190.82 80]
W: Not using locking for read only lock file /var/lib/dpkg/lock-frontend
W: Not using locking for read only lock file /var/lib/dpkg/lock
W: Not using locking for read 

In [20]:
def extract_feature(model: nn.Module, dataloaders: DataLoader, subset: Literal['query', 'gallery'], filename_pattern: str = r'\.') -> np.ndarray:
    '''
    Parameters:
        model: resnet featurizer
        dataloaders: for everything
        subset: to which suibset should the featurization should be restricted
        filename_pattern: matches filename with `re.search`. The default value includes all images
    Return:
        feature vector shape [<number of images in subset, 768]
    '''
    assert isinstance(model, nn.Module)
    assert isinstance(dataloaders, DataLoader)
    assert type(num_query) == int
    with torch.no_grad():
        features = torch.FloatTensor(0, 768).cuda()
        count = 0
        img_path = []
        for data in dataloaders:
            images, _, _, filenames, metadatas = data.values()
            #print('filenames 1', filenames)

            # Select only the images that belong to the desired subset
            subsets: List[Literal['query', 'gallery']] = metadatas['q_or_g']
            mask = torch.tensor([s == subset for s in subsets], dtype=torch.bool)
            images = images[mask]
            filenames = list(np.array(filenames)[mask])
            assert len(images) == sum(1 for s in subsets if s == subset)
            #print('filenames 2', filenames)

            # Select only the images that match the filename pattern
            mask = torch.tensor([bool(re.search(filename_pattern, fn)) for fn in filenames], dtype=torch.bool)
            images = images[mask]
            filenames = list(np.array(filenames)[mask])
            assert len(images) == sum(mask), "Selection count mismatch"

            n, c, h, w = images.size()
            if n==0:
                continue

            count += n
            ff = torch.FloatTensor(n, 768).zero_().cuda()  # 2048 is pool5 of resnet
            for i in range(2):
                input_img = images.cuda()
                outputs = model(input_img)
                f = outputs.float()
                ff = ff + f
            fnorm = torch.norm(ff, p=2, dim=1, keepdim=True)
            ff = ff.div(fnorm.expand_as(ff))
            features = torch.cat([features, ff], 0)
        assert features.shape[1] == 768
        return features.cpu().numpy()

def calculate_params_hash(params: Dict[str, Any]) -> str:
    stringified = json.dumps({k: str(params[k]) for k in params}, sort_keys=True)
    return hashlib.md5(stringified.encode()).hexdigest()

In [25]:
cfg.merge_from_file(config_test)
cfg.freeze()

output_dir = os.path.join(cfg.LOG_ROOT, cfg.LOG_NAME)
if output_dir and not os.path.exists(output_dir):
    os.makedirs(output_dir)

logging.info("Loaded configuration file {}".format(config_test))
with open(config_test, 'r') as cf:
    config_str = "\n" + cf.read()
    logging.info(config_str)
logging.info("Running with config:\n{}".format(cfg))

os.environ['CUDA_VISIBLE_DEVICES'] = cfg.MODEL.DEVICE_ID

model = make_model(cfg, cfg.MODEL.NAME, 0,0,0)
model.load_param(cfg.TEST.WEIGHT)

INFO:root:Loaded configuration file config/UrbanElementsReID_test.yml
INFO:root:
MODEL:
  PRETRAIN_CHOICE: 'imagenet'
  #PRETRAIN_PATH: "../../.cache/torch/hub/checkpoints" # root of pretrain path
  PRETRAIN_PATH: "assets/models" 
  METRIC_LOSS_TYPE: 'triplet'
  IF_LABELSMOOTH: 'on'
  IF_WITH_CENTER: 'no'
  NAME: 'part_attention_vit'
  NO_MARGIN: True
  DEVICE_ID: ('0')
  TRANSFORMER_TYPE: 'vit_base_patch16_224_TransReID'
  STRIDE_SIZE: [16, 16]

INPUT:
  SIZE_TRAIN: [256,128]
  SIZE_TEST: [256,128]
  REA:
    ENABLED: False
  PIXEL_MEAN: [0.5, 0.5, 0.5]
  PIXEL_STD: [0.5, 0.5, 0.5]
  LGT: # Local Grayscale Transfomation
    DO_LGT: True
    PROB: 0.5

DATASETS:
  TRAIN: ('UrbanElementsReID',)
  TEST: ('UrbanElementsReID_test',)
  #ROOT_DIR: ('../../data') # root of datasets
  #ROOT_DIR: '/home/jgf/Desktop/rhome/jgf/baselineChallenge/UrbanElementsReID/'
  ROOT_DIR: 'assets/datasets/urban-reid-challenge'


DATALOADER:
  SAMPLER: 'softmax_triplet'
  NUM_INSTANCE: 4
  NUM_WORKERS: 8

SOLV

using Transformer_type: part token vit as a backbone
using stride: [16, 16], and patch number is num_y16 * num_x8
using drop_out rate is : 0.0
using attn_drop_out rate is : 0.0
using drop_path rate is : 0.1


INFO:PAT.train:Number of parameter: 86.52M


Resized position embedding from size:torch.Size([1, 197, 768]) to size: torch.Size([1, 132, 768]) with height:16 width: 8
Load 153 / 155 layers.
Loading pretrained ImageNet model......from assets/models/jx_vit_base_p16_224-80ecf9dd.pth
===========building our part attention vit===========
Loading trained model from assets/models/PAT/part_attention_vit_60.pth


In [26]:
for testname in cfg.DATASETS.TEST:
    logging.info(f'>>>>>>>>>>>>>>>>>>>>>>> {testname}')
    val_loader, num_query = build_reid_test_loader(cfg, testname)
    if cfg.MODEL.NAME == 'part_attention_vit':
        do_inf_pat(cfg, model, val_loader, num_query)
    else:
        do_inf(cfg, model, val_loader, num_query)

    #logging.info(type(model))
    #logging.info(type(val_loader))
    #logging.info(type(num_query))
num_query_base = int(num_query/(1+number_of_refinements))

INFO:root:>>>>>>>>>>>>>>>>>>>>>>> UrbanElementsReID_test
INFO:root:Dataset and root: UrbanElementsReID_test, and assets/datasets/urban-reid-challenge
INFO:PAT:=> Loaded UrbanElementsReID_test
INFO:PAT:  ----------------------------------------
INFO:PAT:  subset   | # ids | # images | # cameras
INFO:PAT:  ----------------------------------------
INFO:PAT:  query    |     1 |     1384 |         1
INFO:PAT:  gallery  |     1 |     1008 |         3
INFO:PAT:  ----------------------------------------
INFO:PAT.test:Enter inferencing
INFO:PAT.test:Validation Results 
INFO:PAT.test:mAP: 100.0%
INFO:PAT.test:CMC curve, Rank-1  :100.0%
INFO:PAT.test:CMC curve, Rank-5  :100.0%
INFO:PAT.test:CMC curve, Rank-10 :100.0%
INFO:PAT.test:total inference time: 12.05


In [27]:
#qf_base = extract_feature(model, val_loader, subset=r'query')
qf_base = extract_feature(model, val_loader, subset='query', filename_pattern = r'^(?!.*refinement).*$')
qf_A = extract_feature(model, val_loader, subset='query', filename_pattern = r'_refinement_A\.')
qf_B = extract_feature(model, val_loader, subset='query', filename_pattern = r'_refinement_B\.')
qf_C = extract_feature(model, val_loader, subset='query', filename_pattern = r'_refinement_C\.')
gf = extract_feature(model, val_loader, subset='gallery')

logging.info(f'qf_base = {qf_base.shape}')
logging.info(f'qf_A = {qf_A.shape}')
logging.info(f'qf_B = {qf_B.shape}')
logging.info(f'qf_C = {qf_C.shape}')

assert qf_base.shape[0] == num_query_base
assert qf_A.shape[0] == num_query_base
assert qf_B.shape[0] == num_query_base
assert qf_B.shape[0] == num_query_base
assert gf.shape[0] == num_gallery
#np.save("./qf.npy", qf_base)
#np.save("./gf.npy", gf)

INFO:root:qf_base = (346, 768)
INFO:root:qf_A = (346, 768)
INFO:root:qf_B = (346, 768)
INFO:root:qf_C = (346, 768)


In [37]:
#q_g_dist = np.dot(qf_base, np.transpose(gf)) # TODO: This one will have to be calculated 3 times, and averaged out
q_g_dist_base = np.dot(qf_base, np.transpose(gf))
q_g_dist_A = np.dot(qf_A, np.transpose(gf))
q_g_dist_B = np.dot(qf_B, np.transpose(gf))
q_g_dist_C = np.dot(qf_C, np.transpose(gf))
q_g_dist = (base_multiplier*q_g_dist_base + 1*q_g_dist_A + 1*q_g_dist_B + 1*q_g_dist_C)/(base_multiplier+number_of_refinements)

q_q_dist = np.dot(qf_base, np.transpose(qf_base))
g_g_dist = np.dot(gf, np.transpose(gf))

logging.info(f'Query_Gallery_dist = {q_g_dist.shape}')
logging.info(f'Query_Query_dist = {q_q_dist.shape}')
logging.info(f'Galery_Galery_dist = {g_g_dist.shape}')

assert q_g_dist.shape[0] == num_query_base
assert q_g_dist.shape[1] == num_gallery
assert q_q_dist.shape[0] == num_query_base
assert q_q_dist.shape[1] == num_query_base
assert g_g_dist.shape[0] == num_gallery
assert g_g_dist.shape[1] == num_gallery

INFO:root:Query_Gallery_dist = (346, 1008)
INFO:root:Query_Query_dist = (346, 346)
INFO:root:Galery_Galery_dist = (1008, 1008)


In [38]:
re_rank_dist = re_ranking(q_g_dist, q_q_dist, g_g_dist, k1=re_ranking_k1)
indices = np.argsort(re_rank_dist, axis=1)[:, :100]

m, n = indices.shape
# # logging.info('m: {}  n: {}'.format(m, n))
with open(track, 'wb') as f_w:
    for i in range(m):
        write_line = indices[i] + 1
        write_line = ' '.join(map(str, write_line.tolist())) + '\n'
        f_w.write(write_line.encode())


lista_nombres = ["{:06d}.jpg".format(i) for i in range(1, len(indices) + 1)]
output_path = track.split(".txt")[0] + "_submission.csv"

with open(output_path, 'w', newline='') as archivo_csv:
    csv_writter = csv.writer(archivo_csv)
    csv_writter.writerow(['imageName', 'Corresponding Indexes'])
    for numero, track in zip(lista_nombres, indices):
        track_str = ' '.join(map(str, track + 1))
        csv_writter.writerow([numero, track_str])

In [39]:
#!python update.py --config_file {config_test} --track {os.path.join(model_path, "track.txt")}

assert os.path.exists(submission_file_name)
df = pd.read_csv(submission_file_name)
assert df.shape[0] == num_query_base
assert df.shape[1] == 2
logging.info(f'Backuped everything to {experiment_id}')

!cp -r {model_path} {model_path}_backup_{experiment_id}

INFO:root:Backuped everything to 1746094978


In [41]:
submission_message += f"; commit_hash: {subprocess.check_output(['git', 'rev-parse', 'HEAD']).decode('ascii').strip()}"
submission_message += f"; hyperparameters_train_hash: {calculate_params_hash(hyperparams_train)}"
submission_message += f"; hyperparameters_test_hash: {calculate_params_hash(hyperparams_test)}"
submission_message += f"; experiment_ID: {experiment_id}"
logging.info(f'Submitting with message "{submission_message}"')

# Submit the file to the competition
# Uncomment only for actual submissions!
api.competition_submit(submission_file_name, submission_message, competition_name)    

INFO:root:Submitting with message "DSD refinement; commit_hash: 6a355affdb8ee8365325d8dd2aee8df03ea9b715; hyperparameters_train_hash: 8f10df35f3f22a7060923d11debd2f45; hyperparameters_test_hash: 92df54af27bbf9cfde3308cc811d1561; experiment_ID: 1746094978"
100%|██████████| 136k/136k [00:00<00:00, 177kB/s]


{"message": "Successfully submitted to Urban Elements ReID Challenge", "ref": 44436117}